In [1]:
import csv
import os
import json
import numpy as np
import random
import re
import spacy
from tqdm import tqdm

if os.getcwd().split(os.sep)[-1] == 'notebook':
    os.chdir('..')

from data_preprocessing import sample_class_data
from keras.layers import Dense
from keras.models import Sequential
from langid.langid import LanguageIdentifier, model
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_suffix_regex
from typing import List, Any

2022-11-07 14:57:51.379995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 14:57:51.380075: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATA_FOLDER = "./data/"
PREPROCESSED_DATA_PATH = DATA_FOLDER + 'preprocessed_data.json'

# Practical application - Predicting the rating of a movie review
To be able to make a prediction about the rating of a movie review (from bad/0 to very good/5), this notebook will guide you through the steps needed for this use case. To get a better understanding about the different parts of a NLP project, three parts will be looked at in detail: 
1. Data preprocessing
1. Creating datasets for machine learninig tasks
1. Training a model and making predictions


For each part, there is at least one task with two possible approaches depending on your knowledge about NLP and machine learning in general. 

Besides this notebook, there are other additional files and folders that will be used throughout the use case. You do not have to worry about additional imports, everything is already set up for you. The files and folders are:
* data/: folder containing the data that will be used. Please do not make any changes to the content of this folder!
* data_preprocessing.py: Contains the function 'sample_class_data' which will be used for the part 'Training a model and making predictions'. This function balances the distribution of classes in a dataset and can apply oversampling and undersampling

Before you start your work, please make sure that everything is set up correctly (see README.md)!

## Data preprocessing

The goal of this part is to show the steps needed to preprocess data for NLP tasks or machine learning tasks in general. These include
* reading the data
* filtering out unsusable data like empty lines or duplicates
* tokenising, and depending on the use case
    * lemmatising
    * stemming
    * Part of Speech (POS) detection
    * Named Entity Recognition (NER)
* processing classes for predictions (simplyfing, casting to other types, ...)
* saving the processed data into an easy to use format

Important:
Instead of applying your code on all the data (data/filmstarts.tsv), please use the file 'selection_film_reviews.tsv' instead. This contains a small collection of the original dataset with which you can run this part of the use case (for the other parts of the use, an already processed dataset is available). 

In [3]:
# Preparation for the tasks below - loading a spaCy model with a custom tokenizer component, loading the Snowball Stemmer for stemming, 
# and loading LanguageIdentifier for determining the language of a text
de_sm = spacy.load("de_core_news_sm")

def custom_tokenizer(model):
    infix_re = re.compile(r'''[.\,\_\?\:\;\...\‘\’\`\“\”\"\'~]''')
    prefix_re = compile_prefix_regex(model.Defaults.prefixes)
    suffix_re = compile_suffix_regex(model.Defaults.suffixes)

    return Tokenizer(model.vocab, prefix_search=prefix_re.search,
                                  suffix_search=suffix_re.search,
                                  infix_finditer=infix_re.finditer,
                                  token_match=None)

de_sm.tokenizer = custom_tokenizer(de_sm)

snow_stemmer_de = SnowballStemmer('german')

langid_model = LanguageIdentifier.from_modelstring(model, norm_probs=True)

Loading data from a tsv/csv file

In [4]:
# Task: Write a function that loads the content of a .tsv or .csv file (consider the encoding of the file)
def read_csv(path: str, encoding: str = "utf-8", newline: str = "\r\n",
            delimiter:str = ";") -> List:
    """Parses the given CSV and returns an array which contains each row of the
    CSV.

    Args:
        path (str): Path to the CSV which should be read
        encoding (str, optional): The CSV encoding. Defaults to "utf-8".
        newline (str, optional): The newline delimiter. Defaults to "\r\n".
        delimiter (str, optional): The delimiter between each Entry.
            Defaults to ";".

    Returns:
        List: Containing the parsed entries from each row.
    """

    csv_array = []
    
    # start of task
    # TODO open the file
    with open(path, encoding=encoding, newline=newline) as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=delimiter)
        # TODO read content of file
        for row in csv_reader:
            csv_array.append(row)
    # end of task
    return csv_array

Using regex to find and group together repetitions of words

In [5]:
# Task: Write a function that uses regular expressions to find repetitions of a word in a sentence and return said word. If no repetitions have been found, return the input text.

# For the easier version, use the regular expressions from easy_code_help.txt and try to understand them
# Complex version: write the regular expressions yourself

# TODO write regex to find repetitions of a word at the start of a sentence
REPEATER_fast = r"^(.+?).{0,2}\1+?$"
# TODO write regex to group all repetitions of a word
REPEATER_exact = r"^(.+?)(.{0,2}\1)+?$"


def remove_repetitions(s):        
    # start easy task
    # TODO check if input text contains repetitions
    match_fast = re.search(REPEATER_fast, s)
    if match_fast:
        # TODO apply the regex for grouping together reptitions and return repeated word
        match_exact = re.search(REPEATER_exact, s)
        return match_exact.group(1)
    return s
    # end of task

Tokenization, lemmatization, and stemming of movie reviews using spaCy (https://spacy.io/usage/spacy-101#annotations) and nltk's SnowballStemmer (https://www.nltk.org/api/nltk.stem.snowball.html#module-nltk.stem.snowball)

In [6]:
# Task: Write a function that tokenizes, lemmatizes, and applies stemming on an input text using a spaCy model and the Snowball Stemmer. 
def tokenize_and_transform(text: str,
                           spacy_model: Language=de_sm,
                           stemming: bool=False,
                           lemmatization: bool=False,
                           stopword_removal: bool=False,
                           punctuation_removal: bool=False) -> List[str]:
    """Tokenizes and transforms the input text with the use of a spacy model
    and the nltk snowball stemmer.

    Args:
        text (str): The text which should be tokenized and transformed.
        spacy_model (Language): The Spacy model which tokenizes and lemmazes
            the text.  Defaults to de_sm.
        stemming (bool, optional): Tokens are stemmed if set to True. Defaults 
            to False.
        lemmatization (bool, optional): Tokens are lemmatized if set to True.
            Defaults to False.
        stopword_removal (bool, optional): Stopwrod tokens are removed if set 
            to True. Defaults to False.
        punctuation_removal (bool, optional): Punctuation tokens are removed if
            set to True. Defaults to False.

    Returns:
        List[str]: Transformed tokens of the input text
    """
    transformed_text = []
    # defining components of the spaCy model
    docs = spacy_model.pipe([text], disable=["tagger", 
                                             "parser", 
                                             "attribute_ruler", 
                                             "ner"])
    # start of task
    # TODO Iterate over all texts and tokens
    for doc in docs:
        for t in doc:
            # TODO Check if stopword removal should be applied and the current token is a stopword and check if puncuation removal should be applied and if the current token is a punctuation
            if (not (stopword_removal and t.is_stop) and
                not (punctuation_removal and t.is_punct)):
                token = t
                # TODO apply lemmatization if requested 
                if lemmatization:
                    token = token.lemma_
                # TODO apply stemming if requested - use snow_stemmer_de as spaCy does not support stemming
                if stemming:
                    token = snow_stemmer_de.stem(str(token))
                # TODO add token to output if it is not empty and is not '--' or ' '
                if len(str(token)) > 1 and str(token) != '--' and ' ' not in str(token):
                    transformed_text.append(str(token))
    # end of task
    return transformed_text

Main function of the preprocessing task 

In [7]:
# Task: Write a function that iterates over the data and applies the following
# - read each line and add them to a list, skip empty lines
# - determine the language of a line and skip non-German lines
# - Check for word repetitions and group repetitions
# - tokenize and transform each line
# - transform the value of the rating to a data type more usable than String
def preprocess_movie_data(movie_data: List, enable_langid: bool=False) -> List:
    """Standardizes and preprocesses the unprocessed movie data.
    
    Args:
        movie_data (List): Unprocessed movie data
        enable_langid (Bool, optional): Enables the language detection which
            removes foreign languages and unidentifiable texts. This slows down
            the performance significantly. Defaults to False
    Returns:
        List: Preprocessed movie data
    """    
    preprocessed_data = []
    for row in tqdm(movie_data):
        # start of task
        # TODO skip lines with missing content
        if len(row) < 3:
            continue       
        # TODO skip lines with empty values
        if row[0] in [' ', ''] or row[1] in [' ', ''] or row[2] in [' ', '']:
            continue
        # TODO group together the review text if it is contained in more than one column 
        if len(row) > 3:
            row = [row[0], row[1], ' '.join(row[2:])]

        # TODO determine the language of the current line and skip non-German ones using langid_model if requested
        if enable_langid:
            pred_lang = langid_model.classify(row[2]) 
            if  pred_lang[0] != 'de':
                continue
        
        # TODO Check for word repetitions and group repetitions 
        movie_review = remove_repetitions(row[2]).replace("\n", " ")
        # TODO tokenize and transform current line
        tokens = tokenize_and_transform(movie_review,
                                        stemming=True,
                                        lemmatization=True,
                                        stopword_removal=True,
                                        punctuation_removal=True)
        # TODO transform the review rating into a good data format and append formatted line to output as [URL, rating, raw text, transformed text]
        preprocessed_data.append([row[0], int(row[1]), movie_review, tokens])
    # end of task
    return preprocessed_data

Create output JSON file

In [8]:
# Task: Write a function that uses all the functions you worked on above to save the preprocessed data as a .json
def create_preprocessed_dataset(path_to_tsv):
    # TODO read data from .tsv file
    csv_array = read_csv(path_to_tsv,
                         encoding="utf-8",
                         newline="\n",
                         delimiter="\t")

    # TODO preprocess data
    movie_data = preprocess_movie_data(csv_array, enable_langid=True)

    # TODO zip data and save in a data format usable for JSON exports
    url, stars, text, tokens = zip(*movie_data)

    preprocessed_data = {
        'url': list(url),
        'text_transformed': tokens,
        'text': list(text),
        'stars': list(stars)
    }
    
    # TODO Save the data to the file 'data/preprocessed_notebook.json'
    with open(DATA_FOLDER + 'preprocessed_notebook.json', 'w', encoding='utf8') as f:
        json.dump(preprocessed_data, f, ensure_ascii=False)

    # End of task
    return preprocessed_data

In [9]:
exercise_block_1_output = create_preprocessed_dataset(DATA_FOLDER + 'selection_film_reviews.tsv')

100%|██████████| 8/8 [00:00<00:00, 89.66it/s]


## Creating datasets for machine learninig tasks
As for all machine learning tasks, creating datasets for training and evaluation your model is needed. The goal of this part is to create a training, validation, and test dataset with regards to correct formatting.

In [10]:
# Preparation for tasks below

# Loading a spaCy model
de_core = spacy.load("de_core_news_lg", disable=["tagger",
                                                 "morphologizer",
                                                 "parser",
                                                 "lemmatizer",
                                                 "attribute_ruler",
                                                 "ner"])

# loading preprocessed data and their vector representations
data_de_core_vectors = np.load(DATA_FOLDER + "preprocessed_de_core_vecs.npz")
preprocessed_de_core_vectors = dict(data_de_core_vectors)

with open(PREPROCESSED_DATA_PATH, 'r') as f:
    preprocessed_data = json.load(f)

In [11]:
# TODO Depending on what you choose, set use_easy_code to True for easy version, False for complex version
use_easy_code = False

# Task: Write a function that creates lists containing different type of information, see comments for more detail
def format_dataset(processed_data, processed_data_vectors):
    x = [] # contains preprocessed texts
    x_fulltext = [] # contains raw texts
    x_vectors = [] # contains vector representations of texts
    y = [] # contains movie ratings

    # Start of task
    # TODO iterate over the data and append the information of each line to the correct list from above
    for i in range (len(processed_data['text_transformed'])):
        x.append(processed_data['text_transformed'][i])
        x_vectors.append(processed_data_vectors['vectors'][i])
        x_fulltext.append(processed_data['text'][i])
        y.append(processed_data['stars'][i])

    # End of task
    return x, x_fulltext, x_vectors, y

# Task: Create a training, validation, and test dataset with a distribution of 70:15:15 (70% of the data in the training set, and 15% each of the data in the validation and test set)

# Easy version: Use the Python library 'sklearn' to create these sets 
def split_dataset_sklearn_version(x, y, random_seed=42):
    split_data = {}
    # TODO use sklearn to create the train and validation-test set (distribution: 70%-30%)
    x_train, x_val_test, y_train, y_val_test = train_test_split(x,
                                                                y,
                                                                test_size=0.30,
                                                                random_state=random_seed)
    # TODO with x_val_test and y_val_test, use sklearn to create the validation and test set (distribution: 50%-50%)
    x_val, x_test, y_val, y_test = train_test_split(x_val_test,
                                                    y_val_test,
                                                    test_size=0.50,
                                                    random_state=random_seed)
    
    
    split_data = {
        'x_train': x_train,
        'x_val': x_val,
        'x_test': x_test,
        'y_train': y_train,
        'y_val': y_val,
        'y_test': y_test
    }
    return split_data 
# End of easy version

# Complex version: Write a function that creates these datasets without the help of sklearn or any other library that can create suchs sets. 
# Keep in mind that shuffling the data is important!
def split_dataset_code_version(x, y, percent_val, percent_test, random_seed=42):
    split_data = {}
    
    # TODO Check if x and y have the same length and return a ValueError of not
    if len(x) != len(y):
        raise ValueError('X and Y have different lenghts.')

    # TODO generate random seed and shuffle data
    random.seed(random_seed)
    x_y = list(zip(x, y))
    random.shuffle(x_y)
    x, y = zip(*x_y)
    
    # TODO determine the percentage value of the size of the training data 
    percent_train = 1 - percent_val - percent_test
    
    # TODO define indices to access the correct amount of data for training and validation
    train_pos = int(len(x) * percent_train)
    val_pos = int(len(x) * (percent_train + percent_val))
    
    split_data = {
        'x_train': x[:train_pos],
        'x_val': x[train_pos:val_pos],
        'x_test': x[val_pos:],
        'y_train': y[:train_pos],
        'y_val': y[train_pos:val_pos],
        'y_test': y[val_pos:]
    }
    # End of task
    return split_data


x, x_fulltext, x_vectors, y = format_dataset(preprocessed_data, preprocessed_de_core_vectors)

if use_easy_code:
    split_data = split_dataset_sklearn_version(preprocessed_data['text_transformed'],
                            preprocessed_data['stars'])
    task_1_split = split_dataset_sklearn_version(x, y)
    task_1_fulltext_split = split_dataset_sklearn_version(x_fulltext, y)
    task_1_vectors_split = split_dataset_sklearn_version(x_vectors, y)
else:
    split_data = split_dataset_code_version(preprocessed_data['text_transformed'],
                            preprocessed_data['stars'],
                            0.15,
                            0.15)
    task_1_split = split_dataset_code_version(x, y, 0.15, 0.15)
    task_1_fulltext_split = split_dataset_code_version(x_fulltext, y, 0.15, 0.15)
    task_1_vectors_split = split_dataset_code_version(x_vectors, y, 0.15, 0.15)

Transforming the representation of the data using One Hot encoding

In [12]:
num_labels_task_1 = np.max(task_1_split['y_train']) + 1

# functions to apply one hot encoding 
def num_to_one_hot(num, length):
    one_hot = [0]*length
    one_hot[num] = 1
    return one_hot

def y_to_one_hot(y, length):
    return np.array([np.array(num_to_one_hot(e, length)) if isinstance(e, int) else e for e in y])


# Task: Write a function that transforms the data into numpy arrays. 
# For each type of data set, data_split should contain a numpy representation of the text data (x sets) and a one hot encoding representation of the ratings (y sets).
def keras_transform_data(data_split):
    # Start of task

    # TODO create a numpy array for the number of labels
    num_labels = np.max(data_split['y_train']) + 1

    # TODO create numpy arrays for x_train and y_train
    data_split['x_train'] = np.array([np.array(e) if not isinstance(e, np.ndarray) else e for e in data_split['x_train']])
    data_split['y_train'] = y_to_one_hot(data_split['y_train'], num_labels)
    
    # TODO create numpy arrays for x_val and y_val
    data_split['x_val'] = np.array([np.array(e) if not isinstance(e, np.ndarray) else e for e in data_split['x_val']])
    data_split['y_val'] = y_to_one_hot(data_split['y_val'], num_labels)
    
    # TODO create numpy arrays for x_text and y_test
    data_split['x_test'] = np.array([np.array(e) if not isinstance(e, np.ndarray) else e for e in data_split['x_test']])
    data_split['y_test'] = y_to_one_hot(data_split['y_test'], num_labels)
    # End of task
    return data_split

keras_task_1_split = keras_transform_data(task_1_vectors_split)

## Training a model and making predictions
The goal of this part is to create a machine learning model that predicts the rating of a movie review.

Functions for training the model and making predictions

In [13]:
# Task: Using keras, write a function that creates the machine learning model.

# For the easier version, take the code from easy_code_help.txt
def get_model(n_inputs, n_outputs) -> Sequential:
    model = Sequential()
    # TODO create the structure of the Model with three Dense layer and the correct loss function and optimizer
    # TODO add first Dense layer
    model.add(Dense(1000, input_dim=n_inputs, activation='relu'))
    # TODO add second Dense Layer 
    model.add(Dense(1000, activation='relu'))
    # TODO add third Dense Layer
    model.add(Dense(n_outputs, activation='sigmoid'))
    # TODO apply correct loss function and optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Task: Write a function that creates the machine learning model and applies a training and evaluation phase
def train_keras_model(data, batch_size=16, sampling_method="none", epochs=1):
    # Start of task
    model = None
    # TODO define the input and output dimensions of the model and initialize it with this information
    n_inputs, n_outputs = len(data['x_train'][0]), len(data['y_train'][0])
    model = get_model(n_inputs, n_outputs)
    
    # TODO Using the sample_class_data function from data_preprocessing.py, create x_train and y_train with a sampling method
    x_train, y_train = sample_class_data(data['x_train'],
                                         data['y_train'],
                                         method=sampling_method)
    # TODO convert x_train and y_train to numpy arrays
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    # TODO train the model
    model.fit(x_train,
              y_train,
              batch_size=batch_size,
              verbose=1,
              epochs=epochs)
    
    # TODO evaluate the model on the test data set and print out the f1 score of said evaluation (see import function 'from sklearn.metrics import f1_score')
    test_pred = model.predict(data['x_test'])
    print(np.round(f1_score([np.argmax(e) for e in data['y_test']], 
                            [np.argmax(e) for e in test_pred], 
                            average=None), 2))
    model.save(DATA_FOLDER + 'keras_model_movie_reviews')

    print('Model has been saved to data/.')
    
    # End of task
    return model

# Task: Write a function that predicts the rating of a movie review
def predict_movie_rating(model, text) -> Any:
    # TODO create vector representation of the input text usind the spaCy model 'de_core' from further above
    vec = de_core(text).vector
    # TODO make the prediction
    prediction = model.predict(np.array([np.array(vec)]))

    return np.argmax(prediction[0])

In [14]:
# list of movie reviews and their ratings for testing purposes
list_text_stars = [
    ["""Ein Wunderbarer schön geschriebener Film. John Fords zeitloser Klassiker! Auf gleicher Wellenlänge mit dem Buch.""", 5],
    ["""einfach nur hammer. und superlustig. Bin Fan der Reihe und wurde nich enttäuscht. Ganz im Gegenteil.""", 4],
    ["""Realitätsfern, aber nett anzusehen.""", 3],
    ["""Habe den Film heute gesehen, war ziemlich langweilig! Kaum Spannung, absolut vorhersehbar...""",0]
]

model_t1_o_10 = train_keras_model(keras_task_1_split, sampling_method="mediansampling", epochs=10)

2022-11-07 14:58:02.809376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-07 14:58:02.809481: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-07 14:58:02.809501: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (E5790-ANAUG-W10): /proc/driver/nvidia/version does not exist
2022-11-07 14:58:02.809768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
3039/3039 [==============================] - 21s 7ms/step - loss: 1.4998
Epoch 2/10
3039/3039 [==============================] - 21s 7ms/step - loss: 1.4047
Epoch 3/10
3039/3039 [==============================] - 21s 7ms/step - loss: 1.3687
Epoch 4/10
3039/3039 [==============================] - 22s 7ms/step - loss: 1.3409
Epoch 5/10
3039/3039 [==============================] - 22s 7ms/step - loss: 1.3092
Epoch 6/10
3039/3039 [==============================] - 21s 7ms/step - loss: 1.2855
Epoch 7/10
3039/3039 [==============================] - 29s 10ms/step - loss: 1.2615
Epoch 8/10
3039/3039 [==============================] - 30s 10ms/step - loss: 1.2393
Epoch 9/10
3039/3039 [==============================] - 21s 7ms/step - loss: 1.2188
Epoch 10/10
332/332 [==============================] - 1s 3ms/step
[0.35 0.23 0.34 0.44 0.38 0.6 ]
INFO:tensorflow:Assets written to: ./data/keras_model_movie_reviews/assets
Model has been saved to data/.


In [15]:
for movie_review, rating in list_text_stars:
    print(f"Movie review: '{movie_review}', Prediction: {predict_movie_rating(model_t1_o_10, movie_review)}, correct rating: {rating}")
    print('-----------')

1/1 [==============================] - 0s 20ms/step
Movie review: 'Ein Wunderbarer schön geschriebener Film. John Fords zeitloser Klassiker! Auf gleicher Wellenlänge mit dem Buch.', Prediction: 5, correct rating: 5
-----------
1/1 [==============================] - 0s 19ms/step
Movie review: 'einfach nur hammer. und superlustig. Bin Fan der Reihe und wurde nich enttäuscht. Ganz im Gegenteil.', Prediction: 2, correct rating: 4
-----------
1/1 [==============================] - 0s 21ms/step
Movie review: 'Realitätsfern, aber nett anzusehen.', Prediction: 3, correct rating: 3
-----------
1/1 [==============================] - 0s 18ms/step
Movie review: 'Habe den Film heute gesehen, war ziemlich langweilig! Kaum Spannung, absolut vorhersehbar...', Prediction: 0, correct rating: 0
-----------
